In [ ]:
!pip install phoebe

In [ ]:
# - *- coding: utf- 8 - *- 

if __name__ == '__main__':
    import phoebe
    from phoebe import u
    import numpy as np
    import matplotlib.pyplot as plt
    import os

count = 1

cwd = os.getcwd()
path = os.path.join(cwd, "Algol")
try:
    os.mkdir(path)
except:
    print("Algol klasoru zaten var")

logger = phoebe.logger()

arrayPeriod = np.linspace(3, 10, 4)
arrayIncl = np.linspace(70, 90, 4)
arraySma = np.linspace(10, 20, 3)
arrayQ = np.linspace(0.5, 0.8, 3)
arrayTeffP = np.linspace(10000, 13000, 3)
arrayTeffS = np.linspace(5000, 6000, 3)
arrayRequivP = np.linspace(2.79, 2.79, 1)
arrayRequivS = np.linspace(3, 3, 1)

for period in arrayPeriod:
    for incl in arrayIncl:
        for sma in arraySma:
            for q in arrayQ:
                for teffP in arrayTeffP:
                    for teffS in arrayTeffS:
                        for requivP in arrayRequivP:
                            for requivS in arrayRequivS:
                                b = phoebe.default_binary()
                                times = np.linspace(0, period, 100)
                                b.add_dataset('lc', times=times)
                                b["period@binary"] = period * u.day  # Orbital period in t
                                b["incl@binary@orbit@component"] = incl * u.deg
                                b["sma@binary@orbit@component"] = sma * u.solRad  # AU
                                b["q@binary@orbit@component"] = q  # mass ratio
                                b["teff@primary"] = teffP * u.K
                                b["teff@secondary"] = teffS * u.K
                                b["requiv@primary@component"] = requivP * u.solRad
                                b["requiv@secondary@component"] = requivS * u.solRad

                                # Albedo
                                b["irrad_frac_refl_bol@primary@component"] = 1.00  # radiative
                                b["irrad_frac_refl_bol@secondary@component"] = 0.5  # convective
                                # Gravity darkening
                                b["gravb_bol@primary@component"] = 1.00  # radiative
                                b["gravb_bol@secondary@component"] = 0.32  # convective
                                # Let's use linear limb darkening law
                                b["ld_mode_bol@primary"] = "manual"
                                b["ld_mode_bol@secondary"] = "manual"

                                try:
                                    b.run_compute()
                                    model = b["fluxes@latest@model"].get_value()
                                    normflux_model = model / np.average(model)

                                    fig, ax = plt.subplots(nrows=1, ncols=1)
                                    ax.plot(times, normflux_model, "r-")
                                    fig.savefig(cwd + "/Algol/algol" + str(count) + ".png")
                                    plt.close(fig)

                                    file2wr = open(cwd + "/Algol/algol" + str(count) + ".txt", "w")
                                    # str2wr = "time\tflux\n"
                                    # file2wr.write(str2wr)
                                    for i, t in enumerate(times):
                                        str2wr = "{:.6f}\t{:.4f}\n".format(t, normflux_model[i])
                                        file2wr.write(str2wr)
                                    file2wr.close()
                                except:
                                    print("Şu parametrelerle sentetik Algol lc oluşturulamadı: count:" + str(
                                        count) + " period:" + str(period) + " incl:" + str(incl) + " sma:" + str(
                                        sma) + " q:" + str(
                                        q) + " teffP:" + str(teffP) + " teffS:" + str(teffP))

                                count += 1

ModuleNotFoundError: ignored